In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

Found cached dataset kde4 (C:/Users/Raj/.cache/huggingface/datasets/kde4/en-fr-lang1=en,lang2=fr/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [4]:
# this one train dataset needs to be split into train and validation
split_datasets = raw_datasets["train"].train_test_split(test_size=0.1, seed = 20)
split_datasets

Loading cached split indices for dataset at C:\Users\Raj\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-a067345757f9ef23.arrow and C:\Users\Raj\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-22b7e661839e4670.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [5]:
# we can rename the "test" key to "validation" like this
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [6]:
# Let's take a look at one element of the dataset
example = split_datasets["train"][0]
print(example)

{'id': '92924', 'translation': {'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.", 'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."}}


In [8]:
from IPython.display import display, Markdown, JSON
display(Markdown("> **English**\n> {}".format(example["translation"]["en"])))
display(Markdown("> **French**\n> {}".format(example["translation"]["fr"])))

> **English**
> Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.

> **French**
> Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape.

In [11]:
display(JSON(example))

<IPython.core.display.JSON object>

In [12]:
from pprint import pprint
pprint(example)

{'id': '92924',
 'translation': {'en': 'Calibration is about to check the value range your '
                       'device delivers. Please move axis %1 %2 on your device '
                       'to the maximum position. Press any button on the '
                       "device or click on the'Next 'button to continue with "
                       'the next step.',
                 'fr': 'Le calibrage va vérifier la plage de valeurs que votre '
                       "matériel produit. Veuillez déplacer l'axe %1 %2 de "
                       'votre périphérique à la position maximale. Appuyez sur '
                       "n'importe quel bouton du périphérique ou sur le bouton "
                       '« & #160; Suivant & #160; » pour la prochaine étape.'}}


In [14]:
import json
print(json.dumps(example, indent=2, sort_keys=True))

{
  "id": "92924",
  "translation": {
    "en": "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
    "fr": "Le calibrage va v\u00e9rifier la plage de valeurs que votre mat\u00e9riel produit. Veuillez d\u00e9placer l'axe %1 %2 de votre p\u00e9riph\u00e9rique \u00e0 la position maximale. Appuyez sur n'importe quel bouton du p\u00e9riph\u00e9rique ou sur le bouton \u00ab & #160; Suivant & #160; \u00bb pour la prochaine \u00e9tape."
  }
}


In [18]:
JSON(example)

<IPython.core.display.JSON object>

In [16]:
example

{'id': '92924',
 'translation': {'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
  'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."}}

In [19]:
# Testing the behavior of the translation - shows lazy translation - leaves computer science related words in english while translating the rest to french

from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint, tokenizer=model_checkpoint)
translator("Default to expanded threads")

c:\Users\Raj\repos\hf-nlp\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour les threads élargis'}]

In [20]:
# another example of this behavior can be seen with the word plugin
split_datasets["train"][172]["translation"]


{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [21]:
translator(split_datasets["train"][172]["translation"]["en"])

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

In [23]:
# similar behavior with email translation
translator("Write me an email soon")

[{'translation_text': 'Ecrivez-moi un e-mail bientôt'}]

In [24]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

c:\Users\Raj\repos\hf-nlp\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [26]:
# Data prep: remember that the tokenizer processes the target in the output language, here French, so we need to set the source and target languages accordingly
# to see how this work, let's process one sample of each language in the training set
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target = fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [27]:
en_sentence, fr_sentence

('Default to expanded threads',
 'Par défaut, développer les fils de discussion')

In [30]:
# if we forget to indicate that we are tokenizing labels, they will be tokenized by the input tokenizer, which in the case of a Marian model is not going to go well at all

wrong_targets = tokenizer(fr_sentence)
print(wrong_targets)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

{'input_ids': [577, 1205, 483, 13077, 2, 1205, 517, 13926, 1588, 16, 3842, 9, 5, 1710, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [33]:
# Define preprocessing function on the dataset
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, 
                             text_target=targets, 
                             max_length=max_length, 
                             truncation=True)

    return model_inputs

In [34]:
# we can now apply that preprocessing function to our datasets in one go on all the splits
tokenized_datasets = split_datasets.map(preprocess_function, 
                                        batched=True,
                                        remove_columns=split_datasets["train"].column_names,
                                        )

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [36]:
# We will start with the AutoModel to fine tune
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)